# Model recovery attack: `credit-card`

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mra
from mra import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [41]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

def map_letter(a):
    if a not in ["W", "X", "Y", "Z"]:
        return ord(a) - 65
    else:
        return ord(a) - 78

df = pd.read_csv('../datasets/avila.csv', header=None, index_col=None).sample(frac=1)
x_df = df.drop(df.columns[10], axis=1)
min_values = x_df.describe().transpose()['min'].to_numpy()
max_values = x_df.describe().transpose()['max'].to_numpy()
x = x_df.to_numpy()
x = (x-min_values)/(max_values-min_values)
y = np.array([map_letter(a) for a in df[df.columns[10]]]).reshape((len(x), 1)).astype("float32")
train_ds = make_dataset(x, y, lambda t: t)

mra = mra(train_ds)

2022-02-27 23:47:10.302574: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-27 23:47:10.302655: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: xgpd1
2022-02-27 23:47:10.302670: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: xgpd1
2022-02-27 23:47:10.302852: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.91.3
2022-02-27 23:47:10.302918: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.91.3
2022-02-27 23:47:10.302931: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.91.3
2022-02-27 23:47:10.303768: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perfor

In [45]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    output = tf.keras.layers.Dense(64, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    # x = tf.keras.layers.Dense(64, activation="relu")(xin)
    # x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(13, activation="softmax")(xin)
    return tf.keras.Model(xin, output)

In [46]:
log_train = mra.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.sparse_categorical_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=1000
    )

Iteration 0001: Training loss: 2.5574
Iteration 1001: Training loss: 1.4282
Iteration 2001: Training loss: 1.0386
Iteration 3001: Training loss: 1.0004


In [48]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(600, activation=act)(xin)
    x = tf.keras.layers.Dense(300, activation=act)(x)
    # x = tf.keras.layers.Dense(32, activation=act)(x)
    x = tf.keras.layers.Dense(10, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [49]:
log_attack = mra.attack(
    attack_iter=100,
    make_generator=make_generator,
    input_noise="normal",
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.1982, RG-normal: 0.1826, reconstruction validation: 0.0432
Iteration 0002: RG-uniform: 0.2077, RG-normal: 0.1854, reconstruction validation: 0.0310


In [18]:
np.savetxt("./log/credit-4.csv", log_attack, delimiter=",")